In [40]:
stopword_ru = stopwords.words('russian')
len(stopword_ru)

morph = pymorphy2.MorphAnalyzer()

In [42]:
with open('../../data/ml_in_business/stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [43]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [44]:
%%time
#Запускаем очистку текста. Будет долго...
df['msg'] = df['msg'].apply(lambda x: clean_text(x), 1)

<ipython-input-43-7ee348d9b386>:15: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)


Wall time: 1.48 s


In [45]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
df['msg'] = df['msg'].apply(lambda x: lemmatization(x), 1)

Wall time: 18.6 s


А теперь в 3 строчки обучим нашу модель

In [46]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in df['msg'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in df['msg'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [48]:
common_dictionary[11]

'обещать'

In [49]:
%%time
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

Wall time: 18.1 s


In [50]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

In [52]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in df['msg'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

[]


[(0, 0.04),
 (1, 0.04),
 (2, 0.04),
 (3, 0.04),
 (4, 0.04),
 (5, 0.04),
 (6, 0.04),
 (7, 0.04),
 (8, 0.04),
 (9, 0.04),
 (10, 0.04),
 (11, 0.04),
 (12, 0.04),
 (13, 0.04),
 (14, 0.04),
 (15, 0.04),
 (16, 0.04),
 (17, 0.04),
 (18, 0.04),
 (19, 0.04),
 (20, 0.04),
 (21, 0.04),
 (22, 0.04),
 (23, 0.04),
 (24, 0.04)]

In [53]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: машина работать свой строитель вообще тц ставить
topic_1: спасибо нужно нужный дать понять либо большой
topic_2: ребёнок проблема всё помочь двор хозяин счёт
topic_3: говорить утро сегодня неделя водитель смочь правило
topic_4: район наш это балашиха именно информация каждый
topic_5: дом это всё ситуация оплата факт остальной
topic_6: время который мочь рубль второй чтоть иметь
topic_7: весь год видеть хотеть привет платить ждать
topic_8: ещё парковка ситниковый дорога стоить поставить её
topic_9: место найти большой ходить ездить пусть любой
topic_10: квартира собака номер контакт около карта взять
topic_11: чат очень написать личка ремонт дело звонить
topic_12: писать первый интересно туда ехать сидеть несколько
topic_13: делать понимать хотя брать сразу предлагать продавать
topic_14: знать день площадка детский подъезд искать вода
topic_15: это сказать вроде деньга давать смотреть человек
topic_16: москва жить маленький работа кстати лс московский
topic_17: добрый сосед ден

In [54]:
#text = news['title'].iloc[0]

def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [57]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in df['msg'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['msg_id'] = df['msg_id'].values
topic_matrix = topic_matrix[['msg_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

KeyError: 'msg_id'